In [33]:
from data_cleaning import Data_Preprocessing
from arabert.preprocess import ArabertPreprocessor
import pandas as pd
import numpy as np
import torch
import random
from tqdm import tqdm
from scipy.special import softmax
from transformers import RobertaPreTrainedModel,RobertaModel

In [34]:
# Core
import random

# Basics
import numpy as np
import pandas as pd
import torch

# Metrics
from sklearn.metrics import *

# Tokeniser
from transformers import XLMRobertaTokenizer

# Utility
from tqdm import tqdm

# Dataloader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Scheduler
from transformers import get_linear_schedule_with_warmup

# Optimiser
from transformers import AdamW

# Model

import torch.nn as nn
from models import weighted_Roberta


class XLM_Roberta:
    def __init__(self,args):
        # fix the random
        random.seed(args['seed_val'])
        np.random.seed(args['seed_val'])
        torch.manual_seed(args['seed_val'])
        torch.cuda.manual_seed_all(args['seed_val'])
        
        # set device
        self.device = torch.device(args['device'])

        self.weights=args['weights']
        
        # initiliase tokeniser
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', do_lower_case = True)

        self.model_save_path = args['model_save_path']
        self.name = args['name']
        
    ##-----------------------------------------------------------##
    ##----------------- Utility Functions -----------------------##
    ##-----------------------------------------------------------##
    def encode(self,data,max_len):
        input_ids = []
        attention_masks = []
        for sent in tqdm(data):
            # use in-built tokeniser of Bert
            encoded_dict = self.tokenizer.encode_plus(
                            sent,
                            add_special_tokens =True, # for [CLS] and [SEP]
                            max_length = max_len,
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,
#                             return_tensors = 'pt', # return pytorch tensors
            )
            input_ids.append(encoded_dict['input_ids'])
            # attention masks notify where padding has been added 
            # and where is the sentence
            attention_masks.append(encoded_dict['attention_mask'])
            X_data = torch.tensor(input_ids)
            attention_masks_data = torch.tensor(attention_masks)
            
        return [X_data,attention_masks_data]
    
    ##-----------------------------------------------------------##
    ##------------------ Dataloader -----------------------------##
    ##-----------------------------------------------------------##
    def get_dataloader(self,samples, batch_size,is_train=False):
        inputs,masks,labels = samples

        # Convert the lists into tensors.
#         inputs = torch.cat(inputs, dim=0)
#         masks = torch.cat(masks, dim=0)
        labels = torch.tensor(labels)

        # convert to dataset
        data = TensorDataset(inputs,masks,labels)

        if(is_train==False):
            # use random sampler for training to shuffle
            # train data
            sampler = SequentialSampler(data)
        else:
            # order does not matter for validation as we just 
            # need the metrics
            sampler = RandomSampler(data)  

        dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size,drop_last=True)

        return dataloader
    
    ##-----------------------------------------------------------##
    ##----------------- Training Utilities ----------------------##
    ##-----------------------------------------------------------## 
    def get_optimiser(self,learning_rate,model):
        # using AdamW optimiser from transformers library
        return AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8
                )
    
    def get_scheduler(self,epochs,optimiser,train_dl):
        total_steps = len(train_dl) * epochs
        return get_linear_schedule_with_warmup(optimiser, 
                num_warmup_steps = 0, 
                num_training_steps = total_steps)
    
    def evalMetric(self, y_true, y_pred, prefix):
        # calculate all the metrics and add prefix to them
        # before saving in dictionary
        accuracy = accuracy_score(y_true, y_pred)
        mf1Score = f1_score(y_true, y_pred, average='macro')
        f1Score = f1_score(y_true, y_pred)
        area_under_c = roc_auc_score(y_true, y_pred)
        recallScore = recall_score(y_true, y_pred)
        precisionScore = precision_score(y_true, y_pred)

        nonhate_f1Score = f1_score(y_true, y_pred, pos_label=0)
        non_recallScore = recall_score(y_true, y_pred, pos_label=0)
        non_precisionScore = precision_score(y_true, y_pred, pos_label=0)
        return {prefix+"accuracy": accuracy, prefix+'mF1Score': mf1Score, 
            prefix+'f1Score': f1Score, prefix+'auc': area_under_c,
            prefix+'precision': precisionScore, 
            prefix+'recall': recallScore, 
            prefix+'non_hatef1Score': nonhate_f1Score, 
            prefix+'non_recallScore': non_recallScore, 
            prefix+'non_precisionScore': non_precisionScore}
    
    ##-----------------------------------------------------------##
    ##---------------- Different Train Loops --------------------##
    ##-----------------------------------------------------------## 
    def evaluate(self,model,loader,which):
        # to evaluate model on test and validation set

        model.eval() # put model in eval mode

        # maintain total loss to save in metrics
        total_eval_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(loader):
            # separate input, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            with torch.no_grad(): # do not construct compute graph
                outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
            # output is always a tuple, thus we have to 
            # separate it manually
            #loss = outputs[0]
            logits = outputs[0]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float).to(self.device))
            
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))

            # add the current loss
            # loss.item() extracts loss value as a float
            total_eval_loss += loss.item()

            # calculate true labels and convert it into numpy array
            b_y_true = b_labels.cpu().data.squeeze().numpy()
            
            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

        # calculate metrics
        metrics = self.evalMetric(y_true,y_pred,which+"_")

        # Calculate the average loss over all of the batches.
        avg_loss = total_eval_loss / len(loader)
        # add it to the metric
        metrics[which+'_avg_loss'] = avg_loss

        return metrics,y_pred
    
    
    def run_train_loop(self,model,train_loader,optimiser,scheduler):

        model.train() # put model in train mode

        # maintain total loss to add to metric
        total_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(train_loader):
            # separate inputs, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            # Ref: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch#:~:text=In%20PyTorch%20%2C%20we%20need%20to,backward()%20call.
            model.zero_grad()                

            outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

            # outputs is always returned as tuple
            # Separate it manually
            logits = outputs[0]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float).to(self.device))
            
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))
            
            # calculate current loss
            # loss.item() extracts loss value as a float
            total_loss += loss.item()

            # Back-propagation
            loss.backward()

            # calculate true labels
            b_y_true = b_labels.cpu().data.squeeze().numpy()

            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

            # clip gradient to prevent exploding gradient
            # problems
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # gradient descent
            optimiser.step()
            
            # schedule learning rate accordingly
            scheduler.step()

        # calculate avg loss 
        avg_train_loss = total_loss / len(train_loader)

        # calculate metrics
        train_metrics = self.evalMetric(y_true,y_pred,"Train_")
        
        # print results
        print('avg_train_loss',avg_train_loss)
        print('train_f1Score',train_metrics['Train_f1Score'])
        print('train_accuracy',train_metrics['Train_accuracy'])

        # add loss to metrics
        train_metrics['Train_avg_loss'] = avg_train_loss

        return train_metrics
    
    
    ##------------------------------------------------------------##
    ##----------------- Main Train Loop --------------------------##
    ##------------------------------------------------------------##
    def train(self,model,data_loaders,optimiser,scheduler,epochs,save_model):
        # save train stats per epoch
        train_stats = []
        train_loader,val_loader,test_loader = data_loaders
        # maintain best mF1 Score to save best model
        best_mf1Score=-1.0
        for epoch_i in range(0, epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            
            print("")
            print('Training...')
            # run trian loop
            train_metrics = self.run_train_loop(model,train_loader,
                                            optimiser,scheduler)

            print("")
            print("Running Validation...") 
            # test on validation set
            val_metrics,_ = self.evaluate(model,val_loader,"Val")
            
            print("Validation Loss: ",val_metrics['Val_avg_loss'])
            print("Validation Accuracy: ",val_metrics['Val_accuracy'])
            
            stats = {}

            # save model where validation mF1Score is best
            if(val_metrics['Val_mF1Score']>best_mf1Score):
                best_mf1Score=val_metrics['Val_mF1Score']
                if(save_model):
                    torch.save(model.state_dict(), self.model_save_path+
                        '/best_bert_'+self.name+'.pt')
                # evaluate best model on test set
                test_metrics = self.evaluate(model,test_loader,"Test")

            stats['epoch']=epoch_i+1

            # add train and val metrics of the epoch to 
            # same dictionary
            stats.update(train_metrics)
            stats.update(val_metrics)

            train_stats.append(stats)

        return train_stats,test_metrics
    
    ##-----------------------------------------------------------##
    ##----------------------- Main Pipeline ---------------------##
    ##-----------------------------------------------------------##
    def run(self,args,df_train,df_val,df_test):
        # get X and Y data points 
        X_train = df_train['Text'].values
        Y_train = df_train['Label'].values
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values
        X_val = df_val['Text'].values
        Y_val = df_val['Label'].values
        
        # encode data
        # returns list of data and attention masks
        train_data = self.encode(X_train,args['max_len'])
        val_data = self.encode(X_val,args['max_len'])
        test_data = self.encode(X_test,args['max_len'])
        
        # add labels to data so that we can send them to
        # dataloader function together
        train_data.append(Y_train)
        val_data.append(Y_val)
        test_data.append(Y_test)
        
        # convert to dataloader
        train_dl =self.get_dataloader(train_data,args['batch_size'],True)
        val_dl =self.get_dataloader(val_data,args['batch_size'])                          
        test_dl =self.get_dataloader(test_data,args['batch_size'])
        
        # intialise model
        model = weighted_Roberta.from_pretrained(
            'xlm-roberta-base', # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
            params=args['params'],
        )
        model.to(self.device)
        
        optimiser = self.get_optimiser(args['learning_rate'],model)
        
        scheduler = self.get_scheduler(args['epochs'],optimiser,train_dl)
        
        # Run train loop and evaluate on validation data set
        # on each epoch. Store best model from all epochs 
        # (best mF1 Score on Val set) and evaluate it on
        # test set
        train_stats,train_metrics = self.train(model,[train_dl,val_dl,test_dl],
                                optimiser,scheduler,args['epochs'],args['save_model'])
        
        return train_stats,train_metrics
        
    ##-----------------------------------------------------------##
    ##-------------------- Other Utilities ----------------------##
    ##-----------------------------------------------------------##
    def run_test(self,model,df_test,args):
        # to evaluate test set on the final saved model
        # to retrieve results if necessary
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values

        test_data = self.encode(X_test,args['max_len'])

        test_data.append(Y_test)

        test_dl =self.get_dataloader(test_data,32)

        metrics,preds = self.evaluate(model,test_dl,"Test")

        return metrics,preds
    
    def load_model(self,path,args):
        # load saved best model
        saved_model = weighted_Roberta.from_pretrained(
            'xlm-roberta-base', # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
            params=args['params'],
        )
        
        saved_model.load_state_dict(torch.load(path))
        
        return saved_model

In [35]:
args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

In [36]:
def preprocess(df,isArabic):
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [5]:
obj = XLM_Roberta(args)

model_path = "Saved_Models/Shared_Task_eng/best_bert_xlm_roberta_4_all.pt"
saved_model = obj.load_model(model_path,args)
saved_model.to(obj.device)

data_path = "Data_Processed/Shared_Task_eng/all.csv"
df = pd.read_csv(data_path)
df = preprocess(df,False)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [6]:
metrics, preds = obj.run_test(saved_model,df,args)

100%|██████████| 510/510 [01:03<00:00,  8.07it/s]


In [7]:
y_true = df['Label'].values
y_true = y_true[0: len(preds)]

hate_correct = []
hate_incorrect = []
non_hate_correct=[]
non_hate_incorrect=[]

for index in range(len(preds)):
    pred = preds[index]
    true = y_true[index]
    if(pred==true):
        if(true==0):
            non_hate_correct.append(index)
        else:
            hate_correct.append(index)
    else:
        if(true==0):
            non_hate_incorrect.append(index)
        else:
            hate_incorrect.append(index)

In [14]:
samples = random.choices(hate_correct,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

If youre a man women cant be trusted Theyll turn against you its in their nature 16252

Why WomenBoycottTwitter because you act like Ms Whore 14446

DylanFry10 landenmartin Lmao Your stupid quick talking shit that you cant back up and meet me bitch 14022

Exolkaiwife88 EXOpassion  this whore is damn sexy baekhyunsexyass tbh this girl knew how to name her acc  14692

Fahima66819353 your pussy very big maximum 5 cock 15523

5 I was 20 and ostracised from my friendship group and branded a slut because a boy with a girlfriend made unwanted advances on me MeToo 15186

This is bullshit and horseshit

Both this bitch and judge need to be shot and killed 8254

Fuck boy    
 1457

You shouldve ruined her life in any way possible with access to her accounts

Come on dude Bitches who dont act right need to be punished 11619

Thats not really gender specific though I think most men want to settle down with the best woman they can too

In monogamous societies there is less room for different marita

In [15]:
hate_correct_indices = [1184, 7984, 14647, 15538, 8254]

In [22]:
samples = random.choices(hate_incorrect,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

They want our money and peace of mind I say come and get it 6713

Army need man not LGBT 1581

When you first get into a relationship with a crazy girl vs after you get out of it  14710

Hey alauddins maybe you deserve a dick up your ass since your profile picture is a selfie as well   15718

Good decision bipin rawat sir  
Gay will blow the job of male army 2083

Yea and everyone on rfemaledatingstrategy uses men 8469

Arundhati is a traitor and a kutiyaa 1681

When you take her home drunk vs what you wake up to  13539

Comparing contributions of men to those of women reveals that women have made almost no significant contributions in the history of man 13573

Slay the gay 3078



In [23]:
hate_incorrect_indices = [15206, 10100, 5291, 16251, 1681]

In [31]:
samples = random.choices(non_hate_correct,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

feminism ruined marriage and american people hate feminism 3763

Female student Loses marks in essay specifically for using the word mankind because it is sexist according to her feminist professor Dr Anne Scott  This is how pervasive feminism is in university Why would not say education has become feminised 11431

Classic Ed Kempercel  6567

krish veera there is a reason why most of the women who are into feminism
actually go really mad it really triggers the internet is completely filled
with people who has blamed feminism for everything it comes to a point where
your brain gets triggered when you see comments like this  
the other side is that the people who blame feminism end up not listening but
they turn the same comment into different story by narrating it to their
friends saying duuude i said something so logical and this lady in comment
section just went berserk feminists are asshole  
this happens 6000

Just want to say thank you to kobebryant and DukeMLAX for their great wor

In [32]:
non_hate_correct_indices = [3005, 4429, 1884, 3946, 15048]

In [40]:
samples = random.choices(non_hate_incorrect,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

Yes I saw in reality how protective lesbian girl is when it comes to her
girlfriend Salute to that girlslgbt rocks 98

bdotchristian BheeScott Bitch stop tryna be hard hoe and just tell me you love me 15229

How can you not engage As enraging as it all is fuck it all off I say Turn off the power Reset Reboot Escape the cunt booth 14677

D E L E T I N G   M Y   A C C T 12631

I used to be the sweetest girl ever but I met the wrong mfs and they turned me into a bitch Funny how the world can change you 15095

augietwt just stop before you make it worse eat your cock and pussy 14218

The poor kid wanted a fuck and got a bitch prego and now is fucked for life hahahahaha dumb cunts 13674

SUSSANE arundhati roy 285

horrific message and worst portrayal of gay people nobody has the right to
take other persons life whether gay or straight  maybe its cuz the
people behind it want to create a negative propaganda around LGBTQ
community  
  
Totally disappointed  4484

Ive done the same If Im alrea

In [41]:
non_hate_incorrect_indices = [14632, 13173, 13490, 15478, 15095]

In [42]:
def getProbabilities(sentences):
    input_ids = []
    attention_masks = []
    for sent in tqdm(sentences):
        # use in-built tokeniser of Bert
        encoded_dict = obj.tokenizer.encode_plus(
                        sent,
                        add_special_tokens =True, # for [CLS] and [SEP]
                        max_length = 128,
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_tensors = 'pt', # return pytorch tensors
        )
        input_ids.append(encoded_dict['input_ids'])
        # attention masks notify where padding has been added 
        # and where is the sentence
        attention_masks.append(encoded_dict['attention_mask']) 
        
    masks = torch.cat(attention_masks, dim=0)
    inputs = torch.cat(input_ids, dim=0)
    
    with torch.no_grad(): # do not construct compute graph
        outputs = saved_model(inputs, 
                           token_type_ids=None, 
                           attention_mask=masks,
                           labels=None)
    
    logits = outputs[0]
    
    preds = logits.cpu().data.squeeze().numpy()
    
    return softmax(preds,axis=1)

In [47]:
args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cpu',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

obj = XLM_Roberta(args)

model_path = "Saved_Models/Shared_Task_eng/best_bert_xlm_roberta_4_all.pt"
saved_model = obj.load_model(model_path,args)
saved_model.to(obj.device)

data_path = "Data_Processed/Shared_Task_eng/all.csv"
df = pd.read_csv(data_path)
df = preprocess(df,False)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [44]:
import lime
from lime.lime_text import LimeTextExplainer

In [45]:
explainer = LimeTextExplainer(class_names=['non_hate','hate'])

In [49]:
for i,index in enumerate(hate_correct_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/Shared_Task_eng/hate_correct_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 7113.33it/s]


In [50]:
for i,index in enumerate(hate_incorrect_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/Shared_Task_eng/hate_incorrect_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 7880.92it/s]


In [51]:
for i,index in enumerate(non_hate_correct_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/Shared_Task_eng/non_hate_correct_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 7513.98it/s]


In [52]:
for i,index in enumerate(non_hate_incorrect_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/Shared_Task_eng/non_hate_incorrect_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 6166.73it/s]


## Spanish

In [5]:
args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

In [6]:
obj = XLM_Roberta(args)

model_path = "Saved_Models/AMI-Spanish/best_bert_xlm_roberta_1_all.pt"
saved_model = obj.load_model(model_path,args)
saved_model.to(obj.device)

data_path = "Data_Processed/AMI-Spanish/all.csv"
df = pd.read_csv(data_path)
df = preprocess(df,False)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [7]:
metrics, preds = obj.run_test(saved_model,df,args)

100%|██████████| 103/103 [00:12<00:00,  8.05it/s]


In [8]:
y_true = df['Label'].values
y_true = y_true[0: len(preds)]

hate_correct = []
hate_incorrect = []
non_hate_correct=[]
non_hate_incorrect=[]

for index in range(len(preds)):
    pred = preds[index]
    true = y_true[index]
    if(pred==true):
        if(true==0):
            non_hate_correct.append(index)
        else:
            hate_correct.append(index)
    else:
        if(true==0):
            non_hate_incorrect.append(index)
        else:
            hate_incorrect.append(index)

In [10]:
samples = random.choices(hate_correct,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

Mira rolitranco el mamagueva tu eres una puta y eso vas a seguir siendo toda tu maldita y miserable vida y punto y he dicho Fin del comunicado 655

TowandaRebels A m s que sola corregirme puta por punta pero he pasado demasiado rato en Twitter y ya pasa 1557

mwesner1 Te mereces todo lo que est bien que lindo verte as perra  85

suarezfeelings callate perra arrepentida 591

Ojal la Puta sevillista de tu novia te mande donde te mereces antonioadan13 2016

Antonin anda de zorra regalada y seguro hasta hace gritar de placer a su mejor amigo Cario t mereces mucho ms que eso   Porque soy un idiota pero un idiota feliz por el momento Siempre optimista annimo siempre op  1700

laurabozzo Callate la boca vieja loca ven lame mi axila perra 665

detailiant no mereces entrar a mi choza perra 1819

yungpichaku 3adawiyaaa tu eres mi problema puta x 2747

eduardoc60 RuiDeAzevedo411 Rui si tu eres una perra enamorada 21



In [11]:
hate_correct_indices = [1976, 830, 2393, 254, 2016]

In [13]:
samples = random.choices(hate_incorrect,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

BerserkerSaiyan CellMagnifico Tu callate TREMENDA PINCHE PUTA no tienes derecho de opinar porque tu user es una mierda chupa vergas de caballo 2545

Puta wna Cllate Si s pero tu sab po encuentro tierno hasta cuando chocan los nios entre ellos Y esto esto no s es una mezcla de emociones me dej sin palabras po hasta incoherente idk solo quiero que lo abracen   2270

aurelionuno lopezobrador Cllate pendejo de mierda Mentiroso hijo de la gran puta Culero cobarde puto e hijo de tu repinche madre 1707

xSuzuHopkins gimme5yens El de mi polla a granel Que te calles virgen 3272

Tambin te estresa todo hermana y bueno gil callate la puta madre 1193

danieljadue MarianaLaActriz Callate concha de tu madre comunacho hijo de puta que nacio en cuna de oro 1814

nsascast MarHuidobro menuda cara tienessi tienes verguenza cllate la puta boca que menuda mierda has parido tia 2622

ignaciodover comemela hso y si cojemos ahi nomas no pasa nada ahre 1985

JMLugones arielholanDT Independiente Cllate caradura

In [14]:
hate_incorrect_indices = [2530, 2196, 3255, 305, 1193]

In [17]:
samples = random.choices(non_hate_correct,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

41 Me encantan los memes y las tonterias de twitter y Facebook me parto la polla 3083

LA PUTA MADRE TE MERECES TODO LO BUENO florvigna TODOS LOS AOS DEMOSTRAS QUE SOS LA MEJOR  Showmatch2017 MANDA BAILA FLOR AL 13013 2195

GetawayCar78 Hierba mala nunca muere dicen Es un pedazo de imbecil que ojala le cerraran el canal Es el LoganJake paul espaol Escoria 1988

Iros de una puta vez Zidane y Benzema Ya no merecis llevar esa camiseta ni ese escudo 2331

hezmick Y t eres un autntico cateto No hables de lo que no tienes ni puta idea payaso  2740

aaronscoot Laura99CM NiDarrak Es que aunque no lo creas responder eso suena a no llevas razn cllate la puta boca porque no quiero escuchar una verdad aunque no sea lo que quieras expresar 2574

BlueeVeins que te calles la puta boca hombre hecho con el teclado ese que va a voz sabes no se como se llama pero ese 837

Esa gente insultando y condenando al asesino de Diana Quer pero que luego le quitan importancia al acoso violacin y maltrato a la muje

In [18]:
non_hate_correct_indices = [1326, 2133, 615, 770, 2331]

In [19]:
samples = random.choices(non_hate_incorrect,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

Pambisita1568 s0ymia Sucio coges sobra zorra novia jajajajajaj eres muy imbcil wey osea que asco das nunca en tu vida has visto un diccionario o que 659

Que te pasa mardito yo soy puta pq quiero ser puta no pq me pas algo malo Cllate  2858

es una puta gilipollez alarmarse asi enserio que hay de malo no esta desnuda no tiene nada que ensear por que tanto revuelo Para eso puedes poner ropa a tu pj si tanto te molesta verle desnudo un poquito de logica 2668

la familia de mi novia es lo mejor qu hay la puta madre  1062

Tu puta madre pinche chamaco mejor cllate 2078

Te gustara participar en una peli porno buscamos tu perfil una tia guarra con ojos bonitos y no muy guapa  Hostia no te has esforzado mucho sigue intentndolo  1348

AndreiusArdilla Me flipa Me tienes que ensear a hacer pelo me cago en la puta JAJAJA 2765

roman tu eres la puta de vince v RAW 1501

cual es el vehculo de una mujer la aspiradora 835

Slo dentro de los lmites del humor quiz es esta la solucin compensatoria para

In [20]:
non_hate_incorrect_indices = [2858, 1062, 2078, 1501, 835]

In [21]:
args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cpu',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

obj = XLM_Roberta(args)

model_path = "Saved_Models/AMI-Spanish/best_bert_xlm_roberta_1_all.pt"
saved_model = obj.load_model(model_path,args)
saved_model.to(obj.device)

data_path = "Data_Processed/AMI-Spanish/all.csv"
df = pd.read_csv(data_path)
df = preprocess(df,False)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [22]:
import lime
from lime.lime_text import LimeTextExplainer

In [23]:
def getProbabilities(sentences):
    input_ids = []
    attention_masks = []
    for sent in tqdm(sentences):
        # use in-built tokeniser of Bert
        encoded_dict = obj.tokenizer.encode_plus(
                        sent,
                        add_special_tokens =True, # for [CLS] and [SEP]
                        max_length = 128,
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_tensors = 'pt', # return pytorch tensors
        )
        input_ids.append(encoded_dict['input_ids'])
        # attention masks notify where padding has been added 
        # and where is the sentence
        attention_masks.append(encoded_dict['attention_mask']) 
        
    masks = torch.cat(attention_masks, dim=0)
    inputs = torch.cat(input_ids, dim=0)
    
    with torch.no_grad(): # do not construct compute graph
        outputs = saved_model(inputs, 
                           token_type_ids=None, 
                           attention_mask=masks,
                           labels=None)
    
    logits = outputs[0]
    
    preds = logits.cpu().data.squeeze().numpy()
    
    return softmax(preds,axis=1)

In [24]:
explainer = LimeTextExplainer(class_names=['non_hate','hate'])

In [26]:
for i,index in enumerate(hate_correct_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/AMI-Spanish/hate_correct_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 7200.65it/s]


In [28]:
for i,index in enumerate(hate_incorrect_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/AMI-Spanish/hate_incorrect_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 8087.28it/s]


In [29]:
for i,index in enumerate(non_hate_correct_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/AMI-Spanish/non_hate_correct_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 7002.53it/s]


In [30]:
for i,index in enumerate(non_hate_incorrect_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
               a                      num_features=20)
    exp.save_to_file("Outputs_xlm/AMI-Spanish/non_hate_incorrect_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 8360.18it/s]


## Arabic

In [43]:
obj = XLM_Roberta(args)

model_path = "Saved_Models/Let-Mi/best_bert_xlm_roberta_3_all.pt"
saved_model = obj.load_model(model_path,args)
saved_model.to(obj.device)

data_path = "Data_Processed/Let-Mi/all.csv"
df = pd.read_csv(data_path)
df = preprocess(df,True)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [44]:
metrics, preds = obj.run_test(saved_model,df,args)

y_true = df['Label'].values
y_true = y_true[0: len(preds)]

hate_correct = []
hate_incorrect = []
non_hate_correct=[]
non_hate_incorrect=[]

for index in range(len(preds)):
    pred = preds[index]
    true = y_true[index]
    if(pred==true):
        if(true==0):
            non_hate_correct.append(index)
        else:
            hate_correct.append(index)
    else:
        if(true==0):
            non_hate_incorrect.append(index)
        else:
            hate_incorrect.append(index)

100%|██████████| 163/163 [00:20<00:00,  8.06it/s]


In [45]:
samples = random.choices(hate_correct,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

[مستخدم] بلا اخلاق بزيادة عليكي هالكلمة 3341

[مستخدم] يازعرا انت هه 130

[مستخدم] اوقح البشر انت 1451

RT [مستخدم] : داعشية الاعلام نانسي السبع تتبع اساليب الخبث والحقارة في الاعلام 1201

[مستخدم] [مستخدم] انضبي يا شرحوحه 3828

[مستخدم] [مستخدم] # طعمرك شو مسخرة 3519

[مستخدم] واطية رخيصة فاسدة اكتر من معلمينك # ديما _ الواطية 4637

[مستخدم] بتحكي اد فهماتك وانت ما بتفهمي اسكتي احسن 481

[مستخدم] أكلي زباله يا زباله 2235

[مستخدم] نفايات الكترونية و انت وحدة منن 155



In [46]:
hate_correct_indices = [3341,4637, 155, 1201, 481 ]

In [47]:
samples = random.choices(hate_incorrect,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

[مستخدم] قلتلك 100 مرة المطبخ احسن لك . . الولد الشقي بجيب مسبات للعبة # السيد _ نصر _ الله _ ثقتنا 800

[مستخدم] يمنى فواز ، ارحمي حالك ، اشفقي على حالك واسكتي . ما بعرف اذا ممكن اي شخص اشتغل صحافة بيوم من الايام ، اي ول … [رابط] 2630

[مستخدم] ما هي الرسالة التي توجه ؟ سخافة متل سخافة [مستخدم] 128

[مستخدم] [مستخدم] عنجد كان معهن حق يقولوا عنك اشيا ما بتسوا 720

[مستخدم] طز فيكي وبالرينغ قرفتونا بالرينغ وغير الرينج [رابط] 3370

[مستخدم] [مستخدم] [مستخدم] بول البعير كتير عليك 2832

[مستخدم] قلتلك 100 مرة المطبخ احسن لك . . الولد الشقي بجيب مسبات للعبة # السيد _ نصر _ الله _ ثقتنا 800

[مستخدم] لو في دوله بتحترم حالها كنتي انت وأمثالك من صناع الفتن بمستشفى الأمراض العقلية او حبس روميه من زمان . 3165

[مستخدم] ديما بتسوى صرمايه بايام الغلى 4227

[مستخدم] [مستخدم] اي نحن ما منقبلها صرماية بإجرنا . . مبروك . ع راسكم 0



In [48]:
hate_incorrect_indices = [128,720, 800, 3165, 3370]

In [50]:
samples = random.choices(non_hate_correct,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

[مستخدم] شفنا الأعداد يا ست يمنى كلهم على بعضهم ما بيجوا 100 ألف بتسميها انت ثورة ؟ 4222

[مستخدم] اليوم . . بكرا . . بعد شهر . . بعد سنة . . ألف سنة . . ستسقطون ! ! انشاء الله ما تكوني فكرتي كتير قبل ما تكتبيها : D 3821

[مستخدم] اقسم انك اقوى مرا بالكون 2793

كل الحب الكن [مستخدم] [مستخدم] [رابط] 5078

# كلنا _ ديما _ صادق أكيد الجيش الالكتروني السوري ، كلاب # بشار _ الأسد فلتانين و عينبحوا لمساعدة # حسن _ نصر _ الله و حزبو ضد صوت حر متل [مستخدم] 1948

[مستخدم] [مستخدم] [مستخدم] [مستخدم] يا ست ام ديما حضرتك ست محترمة بس ليش ما عرفت تربي ديما 2856

[مستخدم] أخلاقيا سقطوا من زمان وسقطو من عيون الناس 4346

[مستخدم] شكرا ديما صادق من القلب 3223

[مستخدم] اي نسيلنا نصرالله وركزوا عالاساس يرضى عليكي # كلنا _ ديما _ صادق 4513

[مستخدم] انتي بطلة بكل صراحة كل الحب والاحترام 2992



In [51]:
non_hate_correct_indices = [3666, 1721, 476, 501, 4222]

In [54]:
samples = random.choices(non_hate_incorrect,k=10)
for index in samples:
    print(df.iloc[index]['Text'],index)
    print()

[مستخدم] ولا ابوخ من هيك قرطة 5118

[مستخدم] ما انتي وحدة منن نزلتي معن بل انتخابات او نسيتي حالكو قريبة كتير منن ف ما تركبي موجة لعبتك مكشوفة . . و … [رابط] 3167

[مستخدم] [مستخدم] [مستخدم] [مستخدم] احد الممولين لها والداعمين كمشناكي يا آدمية 2808

[مستخدم] خلص ما تزعلي هلق تحسينو بزدلا ياهن شوي قال ثورة قال روحوا انضبوا 3377

[مستخدم] ديما روحي غطي الحدث ب ادلب بيمشي حالك 4164

[مستخدم] انت المصاري فوقك وتحتك شو فارقة معك ؟ ! ! 3938

[مستخدم] [مستخدم] [مستخدم] صاير لسانك طويل الله يلعن الغرور 879

[مستخدم] هني الواطيين حثالة المجتمع ما تهكلي للهم 139

[مستخدم] [مستخدم] يا زلمي ما بتنطاق لا عل الوج ولا عالقفى 1264

[مستخدم] [مستخدم] [مستخدم] [مستخدم] [مستخدم] [مستخدم] نسبية على دايرة وحدة 1043



In [55]:
non_hate_incorrect_indices = [1043, 3075, 576, 2808, 3938]

In [59]:
args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cpu',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

obj = XLM_Roberta(args)

model_path = "Saved_Models/Let-Mi/best_bert_xlm_roberta_3_all.pt"
saved_model = obj.load_model(model_path,args)
saved_model.to(obj.device)

data_path = "Data_Processed/Let-Mi/all.csv"
df = pd.read_csv(data_path)
df = preprocess(df,False)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [60]:
def getProbabilities(sentences):
    input_ids = []
    attention_masks = []
    for sent in tqdm(sentences):
        # use in-built tokeniser of Bert
        encoded_dict = obj.tokenizer.encode_plus(
                        sent,
                        add_special_tokens =True, # for [CLS] and [SEP]
                        max_length = 128,
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_tensors = 'pt', # return pytorch tensors
        )
        input_ids.append(encoded_dict['input_ids'])
        # attention masks notify where padding has been added 
        # and where is the sentence
        attention_masks.append(encoded_dict['attention_mask']) 
        
    masks = torch.cat(attention_masks, dim=0)
    inputs = torch.cat(input_ids, dim=0)
    
    with torch.no_grad(): # do not construct compute graph
        outputs = saved_model(inputs, 
                           token_type_ids=None, 
                           attention_mask=masks,
                           labels=None)
    
    logits = outputs[0]
    
    preds = logits.cpu().data.squeeze().numpy()
    
    return softmax(preds,axis=1)

In [61]:
import lime
from lime.lime_text import LimeTextExplainer

explainer = LimeTextExplainer(class_names=['non_hate','hate'])

In [62]:
for i,index in enumerate(hate_correct_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/Let-Mi/hate_correct_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 8287.17it/s]


In [63]:
for i,index in enumerate(hate_incorrect_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/Let-Mi/hate_incorrect_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 10247.26it/s]


In [65]:
for i,index in enumerate(non_hate_correct_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/Let-Mi/non_hate_correct_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 9281.69it/s]


In [66]:
for i,index in enumerate(non_hate_incorrect_indices):
    exp = explainer.explain_instance(df.iloc[index]['Text'],
                                    getProbabilities, num_samples=100,
                                     num_features=20)
    exp.save_to_file("Outputs_xlm/Let-Mi/non_hate_incorrect_"+str(i+1)+".html")

100%|██████████| 100/100 [00:00<00:00, 8962.38it/s]
